In [ ]:
from lit_bttr import LitBTTR
from PIL import Image
from torchvision.transforms import ToTensor

In [ ]:

import zipfile


with zipfile.ZipFile('crocs-train-zips/test-B/testB.zip', "r") as f:
    fns = f.namelist()
    with f.open('testB/test_00000.jpg', "r") as img_f:
        img = Image.open(img_f).copy()


In [ ]:
img = img.convert('L')

In [ ]:
import numpy as np
np.array(img).shape

In [ ]:
ckpt = 'lightning_logs/version_22/checkpoints/epoch=9-step=17260-val_loss=0.3059.ckpt'
img_path = 'train/train_14339.jpg'
device = 'cuda:1'

In [ ]:
model = LitBTTR.load_from_checkpoint(ckpt)
model.eval()
model.to(device)

In [ ]:
pil_img = Image.open(img_path)
pil_img

In [ ]:
import cv2
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)


In [ ]:
from datamodule import ScaleToLimitRange
from torchvision.transforms import Compose
from torchvision import transforms

transform = Compose([ScaleToLimitRange(), transforms.ToTensor()])
img_trans = transform(img)
img_trans = img_trans.to('cuda:1')



In [ ]:
tgt_str = '\\chemfig { C H O H -[:90] C H O H } + O _ { 2 } \\xrightarrow [ \\triangle ] { A g 或 C u } \\chemfig { C H O -[:90] C H O } + 2 H _ { 2 } O'

In [ ]:
from datamodule import vocab
tgt_indices = vocab.words2indices(tgt_str.split())

In [ ]:
from utils import to_bi_tgt_out

tgt, out = to_bi_tgt_out([tgt_indices], device)


In [ ]:
tgt[0].cpu().numpy()

In [ ]:
vocab.indices2label(tgt[0].cpu().numpy()), vocab.indices2label(tgt[1].cpu().numpy())

In [ ]:
vocab.indices2label(out[0].cpu().numpy())

In [33]:
import torch
out_logits = model.forward(img_trans.unsqueeze(0), torch.ones_like(img_trans), tgt)

In [ ]:
vocab.word2idx['\\Chemfig']

In [ ]:
vocab.indices2label( out_logits.argmax(-1)[0].detach().cpu().numpy())

In [ ]:
' '.join(vocab.indices2label( out_logits.argmax(-1)[1].detach().cpu().numpy()).split()[::-1])

In [46]:
hyp = model.beam_search(img_trans, beam_size=5, max_len=100)

In [ ]:
print(hyp)